# Projet 4 Bernadoy Corentin

Le but de ce projet est de créer une balise html `<a>` autour des textes de référencés dans les différents arrêtés municipaux mis à notre disposition.

## Imports

In [1]:
import os
import bs4
import regex as re

## Stratégie et étude du jeu de donnée

Pour une première approche, nous allons tenter d'utiliser un maximum les expressions régulières. En effet, il s'agit d'une approche simple et si elle réussit, permet de consitituer un premier jeu de données labélisées pour une étude par machine learning. 

Tout d'abord on établit la structure que l'on veut suivre : on va extraire les textes présents dans les balises via Beautifulsoup, recherche la présence de textes de lois dans ces textes extraits, les encadrer par une balise `<a>` puis on va sauvegarder le fichier html modifié dans un fichier séparé.

### Création des dossier de dépôt des résultats

On commence par créer les dossier de résultats dans l'arborescence actuelle

In [58]:
#cellule pour créer les dossiers de résultats
root = "./data/"
os.makedirs(root + "results/", exist_ok= True)
sub_data_dirs = next(os.walk(root + 'data/'))[1]
for sub_data_dir in sub_data_dirs:
    os.makedirs(root + "results/" + sub_data_dir, exist_ok= True)

### Fonctions utiles

Il nous faut trois fonctions pour établir la structure : une première fonction `find_regex` pour évaluer les matchs avec les regex et déterminer leur position dans le texte, une fonction `rewritting` pour réécrire le texte avec les balises et enfin une fonction  `pipeline` qui itère les deux premières sur chaque texte de chaque fichier et qui sauvegarde les résultats

On écrit d'abord la list des regex que l'on va utiliser pour détecter les références à des textes de lois (plus de détails sur comment établir les regexs dans la partie suivante)

In [2]:
regex_list = [
    r"directive[\sA-Za-zÀ-ÿ]+\d{4}/\d+/[A-Z]{2}",
]

In [68]:
### only for tests (if somebody is curious enough to look until here in the commits :))

with open('data/data/0005302750/2011-01-13_APC-auto_pixtral.html', 'r') as f:
    soup = bs4.BeautifulSoup(f, 'html')   
    whole_text = soup.find_all(['div', 'td', 'h2'])
    for tag in whole_text:
        if (not(tag.find('h1'))):
            print(tag)

<div class="dsr-entity">
<div>
     PRÉFECTURE DE L'ORNE
    </div>
<div>
     DIRECTION DE LA RÉGLEMENTATION
    </div>
<div>
     Bureau des Procédures d'Utilité Publique
    </div>
<div>
     NOR : 112C-11-20001
    </div>
</div>
<div>
     PRÉFECTURE DE L'ORNE
    </div>
<div>
     DIRECTION DE LA RÉGLEMENTATION
    </div>
<div>
     Bureau des Procédures d'Utilité Publique
    </div>
<div>
     NOR : 112C-11-20001
    </div>
<div class="dsr-visa">
    VU la directive européenne 2008/105/EC du 16 décembre 2008 établissant des normes de qualité environnementale dans le domaine de l'eau ;
   </div>
<div class="dsr-visa">
    VU la directive européenne 2006/11/CE concernant la pollution causée par certaines substances dangereuses déversées dans le milieu aquatique de la Communauté ;
   </div>
<div class="dsr-visa">
    VU la directive européenne 2000/60/CE du 23 octobre 2000 établissant un cadre pour une politique communautaire dans le domaine de l'eau (DCE) ;
   </div>
<div class="ds

On écrit les fonctions

In [4]:
def find_regex(regex_list, text):
    occurences = []
    for regex in regex_list:
        for occurence in re.finditer(regex, text):
            occurences.append((occurence.start(0),occurence.end(0)))
    return((len(occurences) > 0), occurences)

In [38]:
def rewriting(soup, text, occurences):
    for occ in occurences:
        original_text = text.text
        prefix = original_text[:occ[0]]
        wrapped = original_text[occ[0]:occ[1]]
        suffix = original_text[occ[1]:]

        text.string = ''

        text.append(prefix)

        a_tag = soup.new_tag('a')
        a_tag.string = wrapped
        text.append(a_tag)

        text.append(suffix)

In [48]:
root = "./data/"
def pipeline(root = root):
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
            print(data_path) # à retirer à la fin : uniquement pour du troubleshooting
            with open(data_path, 'r') as f:
                soup = bs4.BeautifulSoup(f, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if (not(tag.find('h1'))): # on exclue le div qui contient le titre, car ce dernier contient toujours le nom du texte présenté dans le document, ce qui n'est pas une référence externe
                        found, occurences = find_regex(regex_list, tag.text)
                        if found:
                            rewriting(soup, tag, occurences)

            with open(res_data_path, 'w', encoding= "utf-8") as f:
                    f.write(str(soup))

In [49]:
pipeline()

./data/data/0003013459/2020-04-20_AP-auto_initial_pixtral.html
False
False
False
False
False
False
./data/data/0003013459/2021-09-24_AP-auto_refonte_pixtral.html
False
False
False
False
False
False
./data/data/0003013459/2023-02-23_APC-auto_pixtral.html
False
False
./data/data/0005205103/2002-12-11_AP-auto_initial_pixtral.html
./data/data/0005205103/2004-04-16_AP-auto_refonte_pixtral.html
./data/data/0005205103/2004-05-04_APC-auto_pixtral.html
./data/data/0005205103/2004-05-04_APC-TAR_pixtral.html
./data/data/0005205103/2006-01-31_AP-audit_pixtral.html
./data/data/0005205103/2007-10-25_AP-auto_refonte_pixtral.html
./data/data/0005205103/2008-08-14_AP-radioactif_pixtral.html
./data/data/0005205103/2008-11-26_AP-audit_pixtral.html
False
./data/data/0005205103/2009-09-11_AP-auto_refonte_pixtral.html
./data/data/0005205103/2010-04-02_AP-MU_pixtral.html
./data/data/0005205103/2011-07-18_AP-RSDE_pixtral.html
False
False
False
False
False
./data/data/0005205103/2012-01-11_AP-auto_refonte_pixt

## Comment établir les regexs :

On écrit une fonction `get_example` qui permet de récupérer toutes les balises contenant un mot de texte de loi à référencer, ce qui nous permet de récupérer toutes les occurences d'un type de loi dans les documents et donc d'avoir une idée de quelle regexp utiliser. On sauvegarde ces textes dans un fichier text séparé afin de pouvoir l'étudier plus facilement.

In [46]:
#### ne pas supprimer : permet de savoir quel format peut avoir chaque texte de loi
def get_examples(Loi):
    root = "./data/"
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    examples = []
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
            with open(data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if((tag.text.find(Loi) != -1)):
                        examples.append(tag.text)

    with open(f"example_{Loi}.txt", 'w') as f:
        for example in examples:
            f.write(f"{example}\n")

In [50]:
get_examples('Code de l’environnement')